# 🔽 Monta o Drive e utiliza o gdown para baixar o arquivo .mp4 diretamente de um link público do Google Drive.

In [12]:
## montar o google driver
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
DATA_DIR = "/content/drive/MyDrive/FASE 4 - Tech Challenge"

In [14]:
import gdown
import shutil
import os

zip_file_id = "1oKzEhA6jzU4YAUmSiN62_J_3-wztmjaC"
zip_output = DATA_DIR

# Use gdown para obter o nome original do arquivo
downloaded_path = gdown.download(
    f'https://drive.google.com/uc?id={zip_file_id}',
    quiet=False,
    output=None  # None mantém o nome original do arquivo do Drive
)

# Move o arquivo baixado para a pasta de destino mantendo o nome original
if downloaded_path is not None:
    nome_arquivo = os.path.basename(downloaded_path)
    caminho_destino = os.path.join(zip_output, nome_arquivo)
    shutil.move(downloaded_path, caminho_destino)
    print(f'Arquivo salvo em: {caminho_destino}')
else:
    print('Download falhou.')

Downloading...
From: https://drive.google.com/uc?id=1oKzEhA6jzU4YAUmSiN62_J_3-wztmjaC
To: /content/Unlocking Facial Recognition_ Diverse Activities Analysis.mp4
100%|██████████| 38.3M/38.3M [00:00<00:00, 152MB/s]


Arquivo salvo em: /content/drive/MyDrive/FASE 4 - Tech Challenge/Unlocking Facial Recognition_ Diverse Activities Analysis.mp4


In [15]:
!pip install deepface opencv-python moviepy tensorflow matplotlib

# Gera um novo vídeo com a marcação das faces e emoções identificadas

In [16]:
import cv2
import numpy as np
from deepface import DeepFace
from moviepy.editor import VideoFileClip, AudioFileClip

# Caminhos de entrada e saída
input_video_path = f"{DATA_DIR}/Unlocking Facial Recognition_ Diverse Activities Analysis.mp4"
output_video_path = f"{DATA_DIR}/output_video_audio_emotion.mp4"

# Inicializa captura de vídeo
cap = cv2.VideoCapture(input_video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define o codec e cria o VideoWriter (sem áudio)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
temp_video_noaudio_path = f"{DATA_DIR}/temp_output_video_noaudio.mp4"
out = cv2.VideoWriter(temp_video_noaudio_path, fourcc, fps, (width, height))

frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

print(f"width: {width}")
print(f"height: {height}")
print(f"fps: {fps}")
print(f"total_frames: {frame_count}")

print("Processando vídeo, aguarde...")
frame_idx = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    try:
        # Detecta todas as faces na imagem
        facesDetect = DeepFace.extract_faces(img_path=frame, detector_backend='retinaface', enforce_detection=True)

        if len(facesDetect) > 0:
          # Detecta faces e analisa emoções
          faces = DeepFace.analyze(frame, actions=['emotion'], detector_backend='retinaface', enforce_detection=False)
          # DeepFace.analyze pode retornar um dict ou uma lista de dicts dependendo do número de faces
          if isinstance(faces, dict):
              faces = [faces]
          for face in faces:
              x, y, w, h = face['region']['x'], face['region']['y'], face['region']['w'], face['region']['h']
              emotion = face['dominant_emotion']
              # Retângulo ao redor da face
              cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
              # Escreve a emoção acima do retângulo
              if(y < 10):
                  cv2.putText(frame, emotion, (x+5, y+20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
              else:
                  cv2.putText(frame, emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    except Exception as e:
        print(f"Erro na detecção em frame {frame_idx}: {e}")

    out.write(frame)
    frame_idx += 1
    if frame_idx % 30 == 0:
        print(f"Frame {frame_idx}/{frame_count}")

cap.release()
out.release()

# Usa moviepy para adicionar o áudio original ao vídeo processado
print("Adicionando áudio ao vídeo final...")
video_clip = VideoFileClip(temp_video_noaudio_path)

audio_path = f"{DATA_DIR}/audio.wav"
audio_clip = AudioFileClip(input_video_path)
audio_clip.write_audiofile(audio_path)

final_clip = video_clip.set_audio(audio_clip)
final_clip.write_videofile(output_video_path, codec="libx264", audio_codec="aac")


width: 1280
height: 720
fps: 30.0
total_frames: 3326
Processando vídeo, aguarde...
Frame 30/3326
Frame 60/3326
Frame 90/3326
Frame 120/3326
Frame 150/3326
Frame 180/3326
Frame 210/3326
Frame 240/3326
Frame 270/3326
Frame 300/3326
Frame 330/3326
Frame 360/3326
Erro na detecção em frame 360: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
Erro na detecção em frame 361: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
Erro na detecção em frame 362: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
Erro na detecção em frame 363: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
Erro na detecção em frame 364: Face could no

MoviePy - Done.
Moviepy - Building video /content/drive/MyDrive/FASE 4 - Tech Challenge/output_video_audio_emotion.mp4.
MoviePy - Writing audio in output_video_audio_emotionTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/FASE 4 - Tech Challenge/output_video_audio_emotion.mp4



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/FASE 4 - Tech Challenge/output_video_audio_emotion.mp4


In [17]:
!pip install openai-whisper

In [18]:
!pip install pysrt googletrans==4.0.0rc1

# Gera as legendas em inglês e português

In [19]:
import whisper

legenda_en_path = f"{DATA_DIR}/legenda_en.srt"
model = whisper.load_model("base")
result = model.transcribe(audio_path, language="en", task="transcribe")
with open(f"{DATA_DIR}/transcription.txt", "w", encoding="utf-8") as f:
    f.write(result["text"])

with open(legenda_en_path, "w", encoding="utf-8") as f:
    for segment in result["segments"]:
        # SRT format: index, time, text
        idx = segment["id"] + 1
        start = segment["start"]
        end = segment["end"]
        # Format time for SRT
        def format_time(t):
            h = int(t // 3600)
            m = int((t % 3600) // 60)
            s = int(t % 60)
            ms = int((t * 1000) % 1000)
            return f"{h:02}:{m:02}:{s:02},{ms:03}"
        f.write(f"{idx}\n{format_time(start)} --> {format_time(end)}\n{segment['text'].strip()}\n\n")


import pysrt
from googletrans import Translator

legenda_pt_path = f"{DATA_DIR}/legenda_pt.srt"

# Carregar legendas originais
subs = pysrt.open(legenda_en_path, encoding='utf-8')
translator = Translator()

# Traduzir cada linha da legenda
for sub in subs:
    translated = translator.translate(sub.text, src='en', dest='pt')  # traduz de inglês para português
    sub.text = translated.text

# Salvar legenda traduzida
subs.save(legenda_pt_path, encoding='utf-8')



# Inclui as legendas no vídeo

In [20]:
import subprocess

output_video_legenda_path = f"{DATA_DIR}/output_video_audio_emotion_com_legendas.mp4"

cmd = [
    "ffmpeg",
    "-i", output_video_path,
    "-vf",
    # Ajuste MarginV conforme o tamanho do vídeo
    f"subtitles={legenda_en_path}:force_style='FontName=Arial,FontSize=15,MarginV=10',subtitles={legenda_pt_path}:force_style='FontName=Arial,FontSize=12,MarginV=40,Outline=1,OutlineColour=&H00808080&'",
    "-c:a", "copy",
    output_video_legenda_path
]

subprocess.run(cmd)

CompletedProcess(args=['ffmpeg', '-i', '/content/drive/MyDrive/FASE 4 - Tech Challenge/output_video_audio_emotion.mp4', '-vf', "subtitles=/content/drive/MyDrive/FASE 4 - Tech Challenge/legenda_en.srt:force_style='FontName=Arial,FontSize=15,MarginV=10',subtitles=/content/drive/MyDrive/FASE 4 - Tech Challenge/legenda_pt.srt:force_style='FontName=Arial,FontSize=12,MarginV=40,Outline=1,OutlineColour=&H00808080&'", '-c:a', 'copy', '/content/drive/MyDrive/FASE 4 - Tech Challenge/output_video_audio_emotion_com_legendas.mp4'], returncode=0)

# Remove arquivos gerados temporariamente

In [21]:
import os

def removeFiles(file_path):

  if os.path.exists(file_path):
      try:
          os.remove(file_path)
          print(f"Arquivo '{file_path}' removido com sucesso do Google Drive.")
      except OSError as e:
          print(f"Erro ao remover o arquivo '{file_path}': {e}")
  else:
      print(f"Arquivo '{file_path}' não encontrado no Google Drive montado.")

removeFiles(temp_video_noaudio_path)
removeFiles(audio_path)
removeFiles(legenda_en_path)
removeFiles(legenda_pt_path)

Arquivo '/content/drive/MyDrive/FASE 4 - Tech Challenge/temp_output_video_noaudio.mp4' removido com sucesso do Google Drive.
Arquivo '/content/drive/MyDrive/FASE 4 - Tech Challenge/audio.wav' removido com sucesso do Google Drive.
Arquivo '/content/drive/MyDrive/FASE 4 - Tech Challenge/legenda_en.srt' removido com sucesso do Google Drive.
Arquivo '/content/drive/MyDrive/FASE 4 - Tech Challenge/legenda_pt.srt' removido com sucesso do Google Drive.
